# Data612 - Project 1

In this first assignment, we’ll attempt to predict ratings with very little information. We’ll first look at
just raw averages across all (training dataset) users. We’ll then account for “bias” by normalizing
across users and across items.

# Movie Recommender System
Our system recommends movies to users. We created a small toy dataset which will be available in github and use it in this recommender system.

### Import required packages

In [40]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

### Load Dataset from Github

In [41]:
movies = pd.read_csv('https://raw.githubusercontent.com/vijay564/DATA612/master/movie_ratings.csv')
movies = movies[sorted(movies.columns)]
movies.head()

,M01,M02,M03,M04,M05,M06,M07,Unnamed: 0
0,5.0,2,4.0,2.0,1.0,3.0,2.0,1
1,3.0,1,1.0,NaN,1.0,1.0,NaN,2
2,NaN,1,3.0,5.0,4.0,NaN,NaN,3
3,4.0,1,3.0,2.0,4.0,2.0,4.0,4
4,NaN,5,NaN,1.0,3.0,4.0,4.0,5


*Finding Mean*

In [42]:
movies.mean(axis=1,skipna = True)

0     2.500000
1     1.500000
2     3.200000
3     3.000000
4     3.666667
5     3.250000
6     3.666667
7     3.625000
8     4.571429
9     3.625000
10    3.571429
11    4.333333
12    4.000000
13    4.142857
14    4.125000
dtype: float64

In [43]:
movies_mean_col = movies.mean(axis=0, skipna = True)
movies_mean_row = movies.mean(axis=1, skipna = True)
print(movies_mean_col)


M01           2.916667
M02           2.400000
M03           3.000000
M04           3.142857
M05           2.714286
M06           2.692308
M07           2.700000
Unnamed: 0    8.000000
dtype: float64


In [44]:
print(movies_mean_row)

0     2.500000
1     1.500000
2     3.200000
3     3.000000
4     3.666667
5     3.250000
6     3.666667
7     3.625000
8     4.571429
9     3.625000
10    3.571429
11    4.333333
12    4.000000
13    4.142857
14    4.125000
dtype: float64


### Split the dataset in 2 datasets : Train and Test

### Find the average of training and test set

In [45]:
movies_col = movies.stack()
movies_col.head(30)
movies_train, movies_test = train_test_split(movies_col, test_size =0.2)

movies_train = movies_train.sort_index()
movies_test = movies_test.sort_index()

movies_train_mean = movies_train.mean()
movies_test_mean = movies_test.mean()

print("Average of training set: " + str(movies_train_mean) + "\n")
print("Average of test set: " + str(movies_test_mean) + "\n")

Average of training set: 3.25

Average of test set: 4.590909090909091



### Calculate the row and column averages to calculate the biases

In [46]:
movies_train_wide = movies_train.unstack().sort_index()
movies_train_column_means = movies_train_wide.mean(axis=0,skipna = True)

print(str(movies_train_column_means))

movies_train_row_means = movies_train_wide.mean(axis=1,skipna = True)

print(str(movies_train_row_means))

M01           3.571429
M02           2.500000
M03           2.833333
M04           3.090909
M05           2.636364
M06           2.666667
M07           2.750000
Unnamed: 0    6.888889
dtype: float64
0     2.600000
1     1.200000
2     3.750000
3     2.666667
4     3.666667
5     3.000000
6     3.400000
7     3.000000
8     4.000000
9     2.600000
10    3.571429
11    5.000000
12    2.714286
13    2.800000
14    4.571429
dtype: float64


In [47]:
movies_test_all = movies_test.unstack().sort_index()
movies_test_all
movies_test_column_means = movies_test_all.mean(axis=0,skipna = True)
print(str(movies_test_column_means))
movies_test_row_means = movies_test_all.mean(axis=1,skipna = True)
print(str(movies_test_row_means))

M01           2.000000
M02           1.000000
M03           5.000000
M04           3.333333
M05           3.000000
M06           3.000000
M07           2.500000
Unnamed: 0    9.666667
dtype: float64
0      2.333333
1      3.000000
2      1.000000
3      4.000000
5      4.000000
6      5.000000
7      5.500000
8      6.000000
9      5.333333
11     1.000000
12    13.000000
13     7.500000
14     1.000000
dtype: float64


### Calculate RMSE

In [48]:
train_pred = [movies_train.mean()]*len(movies_train)
train_rmse = mean_squared_error(movies_train, train_pred)**0.5
print("RMSE training set: " + str(train_rmse) + "\n")
test_pred = [movies_train.mean()]*len(movies_test) 
test_rmse = mean_squared_error(movies_test, test_pred)**0.5
print("RMSE test set: " + str(test_rmse) + "\n")

RMSE training set: 2.3343960165115005

RMSE test set: 3.9593100868252744



### Calculate BIAS for each user and each item.

In [49]:
show_bias = movies_train_column_means - movies_train_mean 
user_bias = movies_train_row_means - movies_train_mean 

### Calculate Baseline Predictors for every user-item combination. 

In [50]:
movies_prediction = pd.DataFrame(index=movies.index, columns=movies.columns.values)
for i in range(0,len(user_bias)):
    for j in range(0,len(show_bias)):
            movies_prediction.iloc[i,j] = movies_train_mean + user_bias[i] + show_bias[j]
            if movies_prediction.iloc[i,j] < 1.0:
                movies_prediction.iloc[i,j] = 1.0
            elif movies_prediction.iloc[i,j] > 5.0:
                movies_prediction.iloc[i,j] = 5.0
movies_prediction.head()

,M01,M02,M03,M04,M05,M06,M07,Unnamed: 0
0,2.92143,1.85,2.18333,2.44091,1.98636,2.01667,2.1,5
1,1.52143,1,1,1.04091,1,1,1,4.83889
2,4.07143,3,3.33333,3.59091,3.13636,3.16667,3.25,5
3,2.9881,1.91667,2.25,2.50758,2.05303,2.08333,2.16667,5
4,3.9881,2.91667,3.25,3.50758,3.05303,3.08333,3.16667,5


In [51]:
movies_prediction_train = pd.DataFrame(index=movies.index, columns=movies.columns.values)
movies_prediction_train = movies_prediction_train[sorted(movies_prediction_train.columns)]
movies_train_wide = movies_train_wide[sorted(movies_train_wide.columns)]
for i in range(0,len(user_bias)):
    for j in range(0,len(show_bias)):
        if math.isnan(movies_train_wide.iloc[i,j]):
            continue
        else:
            movies_prediction_train.iloc[i,j] = movies_train_mean + user_bias[i] + show_bias[j]
            if movies_prediction_train.iloc[i,j] < 1.0:
                movies_prediction_train.iloc[i,j] = 1.0
            elif movies_prediction_train.iloc[i,j] > 5.0:
                movies_prediction_train.iloc[i,j] = 5.0
movies_prediction_train.head()

,M01,M02,M03,M04,M05,M06,M07,Unnamed: 0
0,2.92143,1.85,2.18333,NaN,1.98636,NaN,NaN,5
1,NaN,1,1,NaN,1,1,NaN,4.83889
2,NaN,NaN,3.33333,3.59091,3.13636,NaN,NaN,5
3,2.9881,1.91667,2.25,2.50758,NaN,2.08333,2.16667,NaN
4,NaN,2.91667,NaN,3.50758,3.05303,3.08333,3.16667,5


In [52]:
train_base_rmse = mean_squared_error(movies_train, movies_prediction_train.stack().sort_index())**0.5
print("RMSE training set: " + str(train_base_rmse) + "\n")

RMSE training set: 2.044334238867013



In [53]:
movies_prediction_test = pd.DataFrame(index=movies.index, columns=movies.columns.values)
movies_prediction_test = movies_prediction_test[sorted(movies_prediction_test.columns)]
movies_test_all = movies_test_all[sorted(movies_test_all.columns)]
movies_test_all = movies_test_all.reindex(movies.index)

In [54]:
for l in range(0,movies_test_all.shape[0]):
    for k in range(0,movies_test_all.shape[1]):
        if math.isnan(movies_test_all.iloc[l,k]):
            continue
        else:
            movies_prediction_test.iloc[l,k] = movies_test_mean + user_bias[l] + show_bias[k]
            if movies_prediction_test.iloc[l,k] < 1.0:
                movies_prediction_test.iloc[l,k] = 1.0
            elif movies_prediction_test.iloc[l,k] > 5.0:
                movies_prediction_test.iloc[l,k] = 5.0
movies_prediction_test.head()

,M01,M02,M03,M04,M05,M06,M07,Unnamed: 0
0,NaN,NaN,NaN,3.78182,NaN,3.35758,3.44091,NaN
1,2.86234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,4.34091,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,3.39394,NaN,NaN,5
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### RMSE for Test set

In [55]:
test_base_rmse = mean_squared_error(movies_test, movies_prediction_test.stack().sort_index())**0.5
print("RMSE test set: " + str(test_base_rmse) + "\n")


RMSE test set: 3.4717337115615754



### Percentage improvement for Test and Train set

In [56]:
print("Train: "+str((1-(train_base_rmse/train_rmse))*100) )
print("test: "+str((1-(test_base_rmse/test_rmse))*100) )

Train: 12.425559998939395
test: 12.314680198606432


### Conclusion

> percentage of improvement for test set is 12.31% and train set is 12.42% which is very close. This indicates that after baseline predictor it improved our user-item matrix very slightly.